## 핵심 아이디어
1. 메뉴명을 명사 단위로 토큰화
2. 토큰화한 메뉴명을 빈도분석
3. 빈도 내림차순으로 나열한 뒤 특정 비율의 토큰을 채택
4. 채택한 토큰으로 끝나는 메뉴명을 토큰 카테고리로 분류

## 간단하지만 강력한 알고리즘
대부분 한국어 메뉴명이 분류로 종결된다는 특성을 이용하였습니다  
ex)오곡라떼 => '라떼'라는 분류명으로 종결됨

따라서 이렇게 1차 분류를 시도해보고, 오분류되었거나 미분류된 음식명을 2차 분류해보는건 어떠할지?  

In [ ]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.6/465.6 KB 31.4 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [67]:
# 데이터 임포팅 
df = pd.read_csv('menu_concated_v1.4')
df.dropna(inplace=True)
menu_list = df['menu'].tolist()

In [ ]:
# 명사 단위 토큰화
from konlpy.tag import Okt
from tqdm import tqdm

t = Okt()
menu_list = [t.nouns(word) for word in tqdm(menu_list)]

100%|██████████| 51437/51437 [00:41<00:00, 1227.60it/s]


Okt를 통한 명사 구분의 경우 [자연, 산모, 듬회] 같이 도메인에 맞지 않는 토큰화가 보인다. 

알고리즘의 논리가 명사 분리부터 시작하기 때문에, 이 문제 해결부터 해야한다고 판단한다

2023-01-17

- [해결] 빈도수 비율 없이, 그냥 맨 뒷 토큰을 사용해보는 실험 해보기

- [해결] soynlp를 통한 토큰화 시도

In [ ]:
df

,Unnamed: 0,식당명,메뉴명,메뉴가격,menu
0,0,신풍식당,정식,5000,정식
1,1,부창루,짜장면,5000,짜장면
2,2,부창루,짬뽕,6000,짬뽕
3,3,부창루,우동,6000,우동
4,4,부창루,울면,6000,울면
...,...,...,...,...,...
51432,51432,차이나프로,철판통후추소고기(대/4~5인분),70000,철판통후추소고기
51433,51433,차이나프로,삼선해물누룽지탕(대/4~5인분),50000,삼선해물누룽지탕
51434,51434,차이나프로,비풍당새우(대/4~5인분/저녁메뉴),55000,비풍당새우
51435,51435,차이나프로,파인새우(대/4~5인분),48000,파인새우


## 표준 데이터셋과의 match 비율을 통한 빈도수 상위 %에 대한 임계점 파악하기

In [ ]:
corpus = pd.read_csv('/content/Kor_standard_menu_clean.csv')
text = corpus['title']

In [ ]:
corpus

,Unnamed: 0,title,large_category,category
0,0,케이크 너트 브라우니,빵 및 과자류,케이크
1,1,케이크 녹차바스크치즈케이크,빵 및 과자류,케이크
2,2,케이크 녹차티라미수케이크,빵 및 과자류,케이크
3,3,케이크 뉴욕 치즈케이크,빵 및 과자류,케이크
4,4,케이크 뉴욕슈치즈케이크 (조각),빵 및 과자류,케이크
...,...,...,...,...
8298,8298,청포묵무침,생채·무침류,청포묵무침
8299,8299,치커리겉절이,생채·무침류,치커리겉절이
8300,8300,파래무침 파래 무,생채·무침류,파래무침
8301,8301,짜장소스,"장류, 양념류",짜장소스


In [ ]:
df['tokens'] = menu_list

In [ ]:
from collections import Counter

# Counter 객체는 리스트요소의 값과 요소의 갯수를 카운트 하여 저장하고 있습니다.
# 카운터 객체는 .update 메소드로 계속 업데이트 가능합니다.
word_counts = Counter()

# 토큰화된 각 리뷰 리스트를 카운터 객체에 업데이트 합니다. 
df['tokens'].apply(lambda x: word_counts.update(x))

# 가장 많이 존재하는 단어 순으로 10개를 나열합니다
word_counts.most_common(10)

[('치즈', 1836),
 ('치킨', 1403),
 ('세트', 1367),
 ('추가', 1298),
 ('라떼', 977),
 ('볶음', 850),
 ('밥', 824),
 ('새우', 815),
 ('크림', 796),
 ('탕', 791)]

In [ ]:
len(word_counts)

12046

In [ ]:
std_category = corpus['category'].unique()

In [ ]:
std_category[0]

'케이크'

In [ ]:
# 빈도 상위 10퍼센트의 토큰만 선택
cut_line = int(len(word_counts) / 10)

# 단, 두글자 이상인 토큰만 선택 (1글자 토큰은 결과를 곱창내놓았음;; 하지만 불용어 설정을 잘 한다면??)
outliner_vocab = word_counts.most_common(cut_line)

In [ ]:
int(len(word_counts) * 0.2)

2409

In [ ]:
'치즈' in std_category

True

In [ ]:
def cut_line_test(cut_num):
  # 빈도 상위 cut_num 퍼센트의 토큰만 선택
  cut_line = int(len(word_counts) * cut_num)
  outliner_vocab = word_counts.most_common(cut_line)

  std_match_dict = {}

  for out in outliner_vocab:
    if out[0] in std_category:
      std_match_dict[out[0]] = 1
    else:
      std_match_dict[out[0]] = 0

  match_list = list(std_match_dict.values())
  print(f"match 비율 : {match_list.count(1) / len(match_list)}")

In [ ]:
cut_line_test(0.1)
cut_line_test(0.2)
cut_line_test(0.3)

match 비율 : 0.11627906976744186
match 비율 : 0.0767953507679535
match 비율 : 0.057569886520896764


In [ ]:
cut_line_test(0.4)
cut_line_test(0.5)
cut_line_test(0.6)

match 비율 : 0.0460772104607721
match 비율 : 0.038851070894902875
match 비율 : 0.0326553203265532


In [ ]:
cut_line_test(0.01)
cut_line_test(0.001)
cut_line_test(0.0001)

match 비율 : 0.225
match 비율 : 0.25
match 비율 : 1.0


결론

- match 비율이 상당히 낮다. 즉, 전국5도 데이터에서의 토큰으로는 표준 데이터셋을 설명하기 힘들다는 의미라고 생각할 수도 있다.

- 임계점을 낮출 경우, 전체 범주가 늘어나기 때문에 match 비율이 줄어드는 것은 설명할 수 있다. 하지만, 반대로 임계점을 올렸을 경우에도 match 비율이 1에 가까워져야 하지만 0.25 정도까지만 오르는 것을 보면 결국 표준 데이터셋이 더 설명력이 크다고 볼 수 있다고 판단한다

- 표준 데이터셋을 사전처럼 사용해서 분류하는게 더 정확할 수도 있다는 판단이 들었다

- 빈도수 관련 언어모델의 한계를 생각해보면, 빈도수 비율을 활용한 접근법은 다시 재고해봐야 할 필요성을 느낀다

## '듬' 문제 - 토큰화의 오류 해결

In [ ]:
for i, token_list in enumerate(df['tokens']):
  for token in token_list:
    if '듬' in token:
      print(df['menu'].iloc[i])
      print(token_list)

모듬회
['모듬회']
모듬튀김
['듬', '튀김']
건어물모듬
['건어물', '듬']
모듬
['모듬']
모듬양장피
['듬', '양장', '피']
조개모듬구이
['조개', '듬', '구이']
모듬회스페셜
['듬회', '스페셜']
삼영모듬회스페셜
['듬회', '스페셜']
삼영해산물모듬스페셜
['해산물', '듬', '스페셜']
자연산모듬회
['자연', '산모', '듬회']
모듬해물
['듬', '해물']
해물모듬
['해물', '듬']
해물모듬물회
['해물', '듬', '물회']
살고기모듬수육
['살', '기모', '듬', '수육']
모듬수육
['듬', '수육']
모듬전
['모듬전']
모듬회코스
['듬회', '코스']
한우모듬
['한우', '듬']
모듬양곱창
['듬', '곱창']
모듬스테이크
['듬', '스테이크']
모듬전골
['듬', '전골']
고기듬뿍김치찌개
['고기', '듬뿍', '김치찌개']
해산물모듬
['해산물', '듬']
모듬양념구이
['듬', '양념', '구이']
연어모듬회
['연어', '듬회']
특모듬
['특모듬']
고급어종모듬
['고급', '종', '듬']
모듬조개전복
['듬', '조개', '전복']
모듬조개
['듬', '조개']
생선모듬초밥
['생선', '듬', '초밥']
모듬스페샬
['모듬스페샬']
고기듬북김치찌개
['고기', '듬북', '김치찌개']
모듬오뎅탕
['듬', '오뎅', '탕']
모듬감자튀김
['듬', '감자', '튀김']
순대모듬
['듬']
모듬국밥
['듬', '국밥']
모듬생선구이
['듬', '생선', '구이']
모듬생선회삿뽀로
['듬', '생선회', '삿뽀로']
모듬생선회오타루
['듬', '생선회', '오타루']
모듬생선회오도리
['듬', '생선회', '도리']
모듬초밥개
['듬', '초밥']
모듬꼬치세트
['듬', '꼬치', '세트']
모듬후라이
['듬후', '라이']
모듬회대
['듬회']
모듬회중
['듬', '회중']
모듬회소
['듬', '회소']
돼지고기듬뿍김치찜
['돼지고기', '듬뿍', '김치', '찜']
모듬막창구이
['

토큰화 결과만 봐도 뭔가 이상하다는 것을 알 수 있다.
토크나이저가 도메인에 맞게 훈련되지 않았다는 점이 문제의 원인이라고 가정

표준데이터셋을 학습한 soynlp를 통한 토큰화를 시도해본다

In [1]:
!pip install soynlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 416.8/416.8 KB 10.9 MB/s eta 0:00:00


In [59]:
corpus = pd.read_csv('/content/Kor_standard_menu_clean.csv')

In [11]:
corpus

,Unnamed: 0,title,large_category,category
0,0,케이크 너트 브라우니,빵 및 과자류,케이크
1,1,케이크 녹차바스크치즈케이크,빵 및 과자류,케이크
2,2,케이크 녹차티라미수케이크,빵 및 과자류,케이크
3,3,케이크 뉴욕 치즈케이크,빵 및 과자류,케이크
4,4,케이크 뉴욕슈치즈케이크 (조각),빵 및 과자류,케이크
...,...,...,...,...
8298,8298,청포묵무침,생채·무침류,청포묵무침
8299,8299,치커리겉절이,생채·무침류,치커리겉절이
8300,8300,파래무침 파래 무,생채·무침류,파래무침
8301,8301,짜장소스,"장류, 양념류",짜장소스


In [60]:
corpus = corpus['title']

In [61]:
corpus = corpus.replace(r'\([^)]*\)','',regex=True)

In [62]:
corpus = corpus.drop_duplicates()

In [63]:
from soynlp.tokenizer import MaxScoreTokenizer, RegexTokenizer

tokenizer = RegexTokenizer() ## Regex 기준 토크나이저, 띄어쓰기가 있거나 이어지는 단어가 한국어가 아닐 경우 토큰화

corpus_list = []

for word in corpus:
  corpus_list.append(tokenizer.tokenize(word))

In [ ]:
corpus_list

In [64]:
corpus_dict = {}

for tokens in corpus_list:
  for token in tokens:
    corpus_dict[token] = 1.0

    ## MaxScoreTokenizer에 사용할 커스텀 사전, 점수를 넣을 경우 해당 단어를 토큰으로 분리하는 가중치를 지정
    ## 서울대학교 라는 단어가 있는데, 서울 :1 대학교 : 1 서울대학교 : 1 이처럼 점수가 같을 경우
    ## 단어가 긴 서울대학교를 우선으로 지정해준다

In [ ]:
corpus_dict

In [65]:
tokenizer = MaxScoreTokenizer(scores=corpus_dict) ## 위에서 만든 사전을 점수 계산으로 지정

In [68]:
from tqdm import tqdm

menu_list = [tokenizer.tokenize(word) for word in tqdm(menu_list)]


100%|██████████| 51437/51437 [00:01<00:00, 30892.79it/s]


In [69]:
from collections import Counter

df['tokens'] = menu_list
# Counter 객체는 리스트요소의 값과 요소의 갯수를 카운트 하여 저장하고 있습니다.
# 카운터 객체는 .update 메소드로 계속 업데이트 가능합니다.
word_counts = Counter()

# 토큰화된 각 리뷰 리스트를 카운터 객체에 업데이트 합니다. 
df['tokens'].apply(lambda x: word_counts.update(x))

# 가장 많이 존재하는 단어 순으로 10개를 나열합니다
word_counts.most_common(10)

[('치즈', 1431),
 ('세트', 1372),
 ('치킨', 1358),
 ('새우', 906),
 ('모듬', 644),
 ('볶음', 590),
 ('추가', 570),
 ('라떼', 538),
 ('양념', 536),
 ('샐러드', 529)]

In [70]:
## 가장 적게 존재하는 단어 순으로 정렬합니다. 10개만 나타내는 경우
word_counts.most_common()[:-10:-1] 

[('좌종당계', 1),
 ('비풍당', 1),
 ('철판통후추', 1),
 ('자연송이죽생', 1),
 ('설화샥스핀', 1),
 ('샥스핀홀', 1),
 ('불도장정식', 1),
 ('매운꼬리', 1),
 ('대운탕', 1)]

In [71]:
token_list = word_counts.keys()

In [72]:
len_1_tokens = []

for token in token_list:
  if len(token) <= 1:
    len_1_tokens.append(token)

In [73]:
len(token_list)

24689

In [74]:
len(len_1_tokens)

512

In [28]:
for i, token_list in enumerate(df['tokens']):
  for token in token_list:
    if '듬' in token:
      print(df['menu'].iloc[i])
      print(token_list)

모듬회
['모듬회']
모듬튀김
['모듬튀김']
건어물모듬
['건어물모듬']
모듬
['모듬']
모듬양장피
['모듬', '양장피']
조개모듬구이
['조개모듬구이']
모듬회스페셜
['모듬회스페셜']
삼영모듬회스페셜
['삼영모듬회스페셜']
삼영해산물모듬스페셜
['삼영해산물모듬스페셜']
자연산모듬회
['자연산모듬회']
모듬해물
['모듬해물']
해물모듬
['해물모듬']
해물모듬물회
['해물모듬', '물회']
살고기모듬수육
['살고기모듬', '수육']
모듬수육
['모듬', '수육']
모듬전
['모듬전']
모듬회코스
['모듬회코스']
한우모듬
['한우모듬']
모듬양곱창
['모듬양곱창']
모듬스테이크
['모듬', '스테이크']
모듬전골
['모듬전골']
고기듬뿍김치찌개
['고기듬뿍', '김치찌개']
해산물모듬
['해산물모듬']
모듬양념구이
['모듬양념구이']
연어모듬회
['연어모듬회']
특모듬
['특모듬']
고급어종모듬
['고급어종모듬']
모듬조개전복
['모듬조개전복']
모듬조개
['모듬조개']
생선모듬초밥
['생선모듬', '초밥']
모듬스페샬
['모듬스페샬']
고기듬북김치찌개
['고기듬북', '김치찌개']
모듬오뎅탕
['모듬오뎅탕']
모듬감자튀김
['모듬', '감자튀김']
순대모듬
['순대', '모듬']
모듬국밥
['모듬', '국밥']
모듬생선구이
['모듬생선구이']
모듬생선회삿뽀로
['모듬생선회삿뽀로']
모듬생선회오타루
['모듬생선회오타루']
모듬생선회오도리
['모듬생선회오도리']
모듬초밥개
['모듬', '초밥', '개']
모듬꼬치세트
['모듬꼬치세트']
모듬후라이
['모듬후라이']
모듬회대
['모듬회대']
모듬회중
['모듬회중']
모듬회소
['모듬회소']
돼지고기듬뿍김치찜
['돼지고기', '듬뿍', '김치찜']
모듬막창구이
['모듬', '막창구이']
따로모듬국밥
['따로모듬', '국밥']
모듬구이
['모듬구이']
일돈특수모듬
['일돈특수모듬']
일돈양념모듬
['일돈양념모듬']
특수모듬양념모듬
['특수모듬양념모듬']
양식모듬회
['양식모듬회']
모듬물회
['모듬', '물회'

'듬' 문제는 해결되었다. 커스텀 사전만 잘 다듬고, 더 늘리기만 해도 토큰화 성능이 좋아질거라고 생각한다.

# 빈도수 비율 상관없이 마지막 토큰 = 카테고리

In [ ]:
df['tokens'][0][-1]

'정식'

In [75]:
menu_cluster = {}

for i, tokens in enumerate(df['tokens']):
  menu_cluster[df['menu'][i]] = tokens[-1] ## 빈도수 비율 상관없이, 모든 메뉴명에서 마지막 토큰을 카테고리로 분류

In [76]:
# 분류된 메뉴를 DataFrame화 
df_menu = pd.DataFrame(columns=['title', 'category'], data=menu_cluster.items())
df_menu['category'].unique()

array(['정식', '면', '짬뽕', ..., '설화샥스핀', '자연송이죽생', '좌종당계'], dtype=object)

In [77]:
len(df_menu['category'].unique())

17505

In [ ]:
pd.set_option('display.max_row', 100)

In [ ]:
df_menu.sample(100)

,title,category
31238,고기통만두,만두
33891,앞다리,다리
41301,크림치즈호두김밥,김밥
34922,카츠카레라이스,카레라이스
46590,빅떡갈비김치볶음밥,밥
31102,민족치킨청마치킨,치킨
10432,비빔설록면,비빔설록면
17090,애플티,티
49975,라떼주문시,주문시
17924,마요떡갈비밥버거,버거


In [78]:
custom_dict_find_category = df_menu['category'].value_counts()

In [79]:
custom_dict_find_category.index

Index(['세트', '치킨', '추가', '라떼', '볶음', '덮밥', '샐러드', '김밥', '피자', '떡볶이',
       ...
       '라쥐루즈', '활가리비', '백골뱅이', '미레즈', '파미에피노누아', '펜폴즈맥스쉬라즈', '이기갈지공다스',
       '러더포드힐메를로', '이데딧담', '좌종당계'],
      dtype='object', length=17505)

In [80]:
count_1_list = [custom_dict_find_category.index[i] for i, x in enumerate(custom_dict_find_category) if x == 1]

In [96]:
count_1_list_menu = [df_menu[df_menu['category'] == x]['title'].iloc[0] for x in count_1_list]

In [97]:
df_count_1 = pd.DataFrame()

In [98]:
df_count_1['title'] = count_1_list_menu
df_count_1['category'] = count_1_list

In [99]:
df_count_1

,title,category
0,해초고등어해초추가시,해초고등어해초추가시
1,후라이드오뎅탕,후라이드오뎅탕
2,소꼬리찜,소꼬리찜
3,우신,우신
4,생둥심,생둥심
...,...,...
14619,펜폴즈맥스쉬라즈,펜폴즈맥스쉬라즈
14620,이기갈지공다스,이기갈지공다스
14621,러더포드힐메를로,러더포드힐메를로
14622,이데딧담,이데딧담


In [101]:
df_count_1.to_csv("back_token_count_1.csv")

In [83]:
df_menu[df_menu['category'] == '좌종당계']

,title,category
51436,좌종당계,좌종당계


### 생각할 부분

- 단어 사전에 없을 경우, 그냥 메뉴명 전체가 카테고리가 된다.

- 메뉴명은 결국 한정되어 있고, 표준화를 위해서는 어느정도 수작업이 필요한데 커스텀 사전을 우리가 직접 넣는다면?

- 현재 내 판단으로는 위의 MaxScoreTokenizer에 넣을 커스텀 사전을 수작업을 통해 넣는게 좋다는 생각이 든다.

# 핵심 아이디어
1. 이전 토큰 종결 분류에서 계속된다  

2. 특정 토큰과 가장 가까운 토큰을 몇 개 추린다  
ex) 해장국 => 뼈, 콩나물  

3. 특정 토큰으로 분류된 음식명에 가까운 토큰이 포함되어 있나 조사한다  
ex) 뼈다귀 해장국 => 해장국으로 분류됨 + 뼈 토큰 포함  

4. 그럼 그 음식명에 속성을 추가해준다  
ex) 뼈다귀 해장국 => 해장국 속성 + 뼈 속성  

5. 두 속성이 동시에 포함된 음식명을 전체 음식 목록에서 찾는다  
ex) 해장국과 뼈가 동시에 포함된 음식명 = 뼈다귀 해장국  

6. 세부 분류를 한다  
ex) 뼈해장국을 해장국 > 뼈해장국으로 분류 


In [ ]:
# 국밥을 예시로 시도해본다
df_menu[df_menu['category'] == '국밥']

,title,category
19520,특미마라국밥,국밥
19521,시래기순대국밥,국밥
19522,매생이국밥,국밥
19523,아우내국밥,국밥
19524,얼큰차돌국밥,국밥
...,...,...
19709,선지국밥,국밥
19710,수구레국밥,국밥
19711,송이국밥,국밥
19712,얼큰순대국밥,국밥


In [ ]:
# 국밥 리스트 생성 
gookbab = df_menu[df_menu['category']=='국밥']['title'].tolist()
gookbab[:10]

['특미마라국밥',
 '시래기순대국밥',
 '매생이국밥',
 '아우내국밥',
 '얼큰차돌국밥',
 '순두부국밥',
 '장터따로국밥',
 '하노이국밥',
 '내장섞어국밥',
 '한우우거지국밥']

In [ ]:
# 국밥 명사단위로 토큰화 후 리스트 내 리스트 제거 
gookbab_tokenized = [t.nouns(word) for word in gookbab]
gookbab_tokenized = [w for word in gookbab_tokenized for w in word]
gookbab_tokenized[:10]

['특', '국밥', '시래기', '순대', '국밥', '생', '국밥', '아우', '국밥', '얼']

In [ ]:
# 토큰화된 국밥 리스트 분석
import nltk
gookbab_ko = nltk.Text(gookbab_tokenized, name='국밥 토큰')

In [ ]:
# 국밥 토큰 개수와 빈도 분석 
gookbab_similar = gookbab_ko.vocab().most_common(15)
len(gookbab_ko), gookbab_similar

(446,
 [('국밥', 164),
  ('순대', 19),
  ('콩나물', 18),
  ('얼', 15),
  ('한우', 9),
  ('소고기', 9),
  ('장터', 8),
  ('굴국밥', 8),
  ('김치', 7),
  ('장국밥', 7),
  ('돼지국밥', 7),
  ('시래기', 5),
  ('우거지', 5),
  ('고기', 5),
  ('소머리국밥', 5)])

In [ ]:
# 국밥 종류 생성
kind_of_gookbab = [value for value, num in gookbab_similar if len(value) != 1]
kind_of_gookbab

['국밥',
 '순대',
 '콩나물',
 '한우',
 '소고기',
 '장터',
 '굴국밥',
 '김치',
 '장국밥',
 '돼지국밥',
 '시래기',
 '우거지',
 '고기',
 '소머리국밥']

In [ ]:
for token in kind_of_gookbab[1:]: #kind_of_gookbab[0] = '국밥'
  for menu in gookbab:
    if token in menu:
      print(f'{menu}: {token}')

시래기순대국밥: 순대
부산순대국밥: 순대
순대고기국밥: 순대
순대국밥: 순대
매운고추순대국밥: 순대
순대만국밥: 순대
김치순대국밥: 순대
따로순대국밥: 순대
순대뽈살국밥: 순대
올순대국밥: 순대
토종순대국밥: 순대
우거지순대국밥: 순대
황태순대국밥: 순대
콩나물순대국밥: 순대
특매운순대국밥: 순대
찰순대국밥: 순대
순대지옥국밥: 순대
얼큰장순대국밥: 순대
사골순대국밥: 순대
고기순대섞어국밥: 순대
흰순대국밥: 순대
한우사골순대국밥: 순대
얼큰이순대국밥: 순대
매운순대국밥: 순대
특순대국밥: 순대
순대국국밥: 순대
장순대국밥: 순대
살고기순대국밥: 순대
순대섞어국밥: 순대
얼큰순대국밥: 순대
양지콩나물국밥: 콩나물
김치콩나물따로국밥: 콩나물
콩나물김치국밥: 콩나물
한우콩나물국밥: 콩나물
불고기콩나물국밥: 콩나물
황태콩나물국밥: 콩나물
전주콩나물국밥: 콩나물
콩나물따로국밥: 콩나물
콩나물순대국밥: 콩나물
어린이콩나물국밥: 콩나물
전주식콩나물국밥: 콩나물
김치콩나물국밥: 콩나물
굴콩나물국밥: 콩나물
콩나물국밥: 콩나물
따로굴콩나물국밥: 콩나물
전주끊이는식콩나물국밥: 콩나물
전주남부시장식콩나물국밥: 콩나물
콩나물황태해장국밥: 콩나물
한우우거지국밥: 한우
한우콩나물국밥: 한우
한우장국밥: 한우
한우소머리국밥: 한우
한우소국밥: 한우
한우사골순대국밥: 한우
한우장터국밥: 한우
한우얼큰소머리국밥: 한우
한우따로국밥: 한우
한우시골장터국밥: 한우
한우국밥: 한우
구월산소고기국밥: 소고기
얼큰소고기국밥: 소고기
소고기국밥: 소고기
연변소고기국밥: 소고기
소고기따로국밥: 소고기
장터소고기국밥: 소고기
양평소고기국밥: 소고기
한끼소고기국밥: 소고기
얼큰소고기가마솥국밥: 소고기
장터따로국밥: 장터
소뼈장터국밥: 장터
한우장터국밥: 장터
장터소고기국밥: 장터
장터국밥: 장터
한우시골장터국밥: 장터
시골장터국밥: 장터
장터소국밥: 장터
들깨굴국밥: 굴국밥
하얀굴국밥: 굴국밥
따로매생이굴국밥: 굴국밥
굴국밥: 굴국밥
매생이굴국밥: 굴국밥
통영굴국밥: 굴국밥
얼큰굴국밥: 굴국밥
따로굴국밥